In [1]:
import dlib
import cv2
import sys
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

#tkinter 라이브러리를 사용하여 창을 생성
main = tk.Tk()
main.title("Final_Free Project")

#영상 창의 레이아웃 설정
cvFrame = tk.Frame(main)
cvFrame.grid(row = 0, column = 0, columnspan=6, padx = 10, pady = 10)

lbl1 = tk.Label(cvFrame)
lbl1.grid(row = 0, column = 0, columnspan=6)


#click 변수를 0으로 초기화
click = 0

#얼굴 검출을 위한 검출 파일 불러옴
faceDetector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml') 


#얼굴 검출을 이용하여 얼굴 모자이크 필터 함수 구현 
def Mosaic(image):
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        # draw rect on detected face
        x1 = faceRects[i].left()
        y1 = faceRects[i].top()
        x2 = faceRects[i].right()
        y2 = faceRects[i].bottom()
        
        image_roi = image[y1:y1+150, x1:x1+150] 
        rh, rw = image_roi.shape[:2]
        output = cv2.GaussianBlur(image_roi, (0,0), 8.0)
        image[y1:y1+150, x1:x1+150] = output
        
    return image

#화면 전체를 가우시안블러를 이용하여 스케치 필터 함수 구현
def Sketch(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    canny_edges = cv2.Canny(img_gray_blur, 15, 70) 
    # Do an invert binarize the image 
    ret, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    
    return mask

#랜드마크를 통해 눈 위치를 검출하여 눈 부분에 슈프림 스티커 함수 구현
def Supreme(image):
    sun_image = cv2.imread('supreme_logo.png')
    h, w = image.shape[:2]
    
    image_gray = cv2.cvtColor(sun_image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(image_gray, 200, 255, cv2.THRESH_BINARY_INV)
    mask = sun_image
    mask_image = cv2.bitwise_and(sun_image, mask)
    sh, sw = mask_image.shape[:2]

    zero_mask = np.zeros_like(image)
    zero_mask_image = np.zeros_like(image)

    rx_x = np.int(sw*0.15); lx_x = np.int(sw*(1-0.13))
    eye_y = np.int(sh*0.45)
    eyedot = [(rx_x, eye_y), (lx_x, eye_y)]
    
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        landmarks = landmarkDetector(image, faceRects[i])
        rightp = (landmarks.part(36).x, landmarks.part(36).y) # right eye
        leftp = (landmarks.part(45).x, landmarks.part(45).y) # left eye
        eyedotDst = [rightp, leftp]

        ret = cv2.estimateAffinePartial2D(np.array([eyedot]), np.array([eyedotDst]))
        xform = ret[0]

        xformed_mask = cv2.warpAffine(mask, xform, (w,h))
        xformed_mask_img = cv2.warpAffine(mask_image, xform, (w, h))

        zero_mask = zero_mask + xformed_mask
        zero_mask_image = zero_mask_image + xformed_mask_img
        
    image_blend = (image*0.0 + zero_mask_image*1.0).astype(np.uint8)
    zero_mask_float = zero_mask / 255
    zero_mask_invert = 1 - zero_mask
    image_object = (image_blend * zero_mask_float).astype(np.uint8)
    image_bg = (image * (1-zero_mask_float)).astype(np.uint8)
    image_final = image_object + image_bg
    
    return image_final

#눈과 얼굴 검출로 얼굴 위치에 좀비 마스크 함수를 구현
def Zombie(image):
    sun_image = cv2.imread('zombie.png')
    h, w = image.shape[:2]
    
    image_gray = cv2.cvtColor(sun_image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(image_gray, 200, 255, cv2.THRESH_BINARY_INV)
    #mask = sun_image
    mask = cv2.merge((mask,mask,mask))
    mask_image = cv2.bitwise_and(sun_image, mask)
    sh, sw = mask_image.shape[:2]

    zero_mask = np.zeros_like(image)
    zero_mask_image = np.zeros_like(image)

    rx_x = np.int(sw*0.38); lx_x = np.int(sw*(1-0.36))
    eye_y = np.int(sh*0.48)
    eyedot = [(rx_x, eye_y), (lx_x, eye_y)]
    
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        landmarks = landmarkDetector(image, faceRects[i])
        rightp = (landmarks.part(36).x, landmarks.part(36).y) # right eye
        leftp = (landmarks.part(45).x, landmarks.part(45).y) # left eye
        eyedotDst = [rightp, leftp]

        ret = cv2.estimateAffinePartial2D(np.array([eyedot]), np.array([eyedotDst]))
        xform = ret[0]

        xformed_mask = cv2.warpAffine(mask, xform, (w,h))
        xformed_mask_img = cv2.warpAffine(mask_image, xform, (w, h))

        zero_mask = zero_mask + xformed_mask
        zero_mask_image = zero_mask_image + xformed_mask_img
        
    image_blend = (image*0.0 + zero_mask_image*1.0).astype(np.uint8)
    zero_mask_float = zero_mask / 255
    zero_mask_invert = 1 - zero_mask
    image_object = (image_blend * zero_mask_float).astype(np.uint8)
    image_bg = (image * (1-zero_mask_float)).astype(np.uint8)
    image_final = image_object + image_bg
    
    return image_final

# 검출 후 위치 조정을 통해 코로나마스크 필터 함수 구현
def Corona(image):
    sun_image = cv2.imread('corona_mask.png')
    h, w = image.shape[:2]
    
    image_gray = cv2.cvtColor(sun_image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(image_gray, 250, 255, cv2.THRESH_BINARY_INV)
    mask = cv2.merge((mask,mask,mask))
    mask_image = cv2.bitwise_and(sun_image, mask)
    sh, sw = mask_image.shape[:2]

    zero_mask = np.zeros_like(image)
    zero_mask_image = np.zeros_like(image)

    rx_x = np.int(sw*0.38); lx_x = np.int(sw*(1-0.34))
    eye_y = np.int(sh*0.27)
    eyedot = [(rx_x, eye_y), (lx_x, eye_y)]
    
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        landmarks = landmarkDetector(image, faceRects[i])
        rightp = (landmarks.part(38).x, landmarks.part(38).y) # right eye
        leftp = (landmarks.part(45).x, landmarks.part(45).y) # left eye
        eyedotDst = [rightp, leftp]

        ret = cv2.estimateAffinePartial2D(np.array([eyedot]), np.array([eyedotDst]))
        xform = ret[0]

        xformed_mask = cv2.warpAffine(mask, xform, (w,h))
        xformed_mask_img = cv2.warpAffine(mask_image, xform, (w, h))

        zero_mask = zero_mask + xformed_mask
        zero_mask_image = zero_mask_image + xformed_mask_img
        
    image_blend = (image*0.0 + zero_mask_image*1.0).astype(np.uint8)
    zero_mask_float = zero_mask / 255
    zero_mask_invert = 1 - zero_mask
    image_object = (image_blend * zero_mask_float).astype(np.uint8)
    image_bg = (image * (1-zero_mask_float)).astype(np.uint8)
    image_final = image_object + image_bg
    
    return image_final


#해당 필터 함수가 실행할 때 click의 변수 값 설정
def faceCorona():
    global click; click = 5
    
def faceZombie():
    global click; click = 4
    
def faceSupreme():
    global click; click = 3
    
def faceMosaic():
    global click; click = 2
    
def SketchEffect():
    global click; click = 1


# 윈도우 창에서 필터를 변환해줄 버튼을 구현 
btn1 = tk.Button(cvFrame, text = "Sketchify", font = ('Arial', '20', 'bold'), foreground="Black", command = SketchEffect)
btn1.grid(row = 1, column = 0, sticky = tk.N + tk.S + tk.W + tk.E)

btn2 = tk.Button(cvFrame, text = "Face Mosaic", font = ('Arial', '20', 'bold'), foreground="Green", command = faceMosaic)
btn2.grid(row = 1, column = 1, sticky = tk.N + tk.S + tk.W + tk.E)

btn3 = tk.Button(cvFrame, text = "Supreme", font = ('Arial', '20', 'bold'), foreground="Red", command = faceSupreme)
btn3.grid(row = 1, column = 2, sticky = tk.N + tk.S + tk.W + tk.E)

btn4 = tk.Button(cvFrame, text = "Zombie", font = ('Arial', '20', 'bold'), foreground="Orange", command = faceZombie)
btn4.grid(row = 1, column = 3, sticky = tk.N + tk.S + tk.W + tk.E)

btn5 = tk.Button(cvFrame, text = "Corona", font = ('Arial', '20', 'bold'), foreground="Magenta", command = faceCorona)
btn5.grid(row = 1, column = 4, sticky = tk.N + tk.S + tk.W + tk.E)

# 영상을 띄울 실시간 웹캠 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('frame_size = ', frame_size)


# click이 해당 숫자일 시 해당 함수를 실행시켜 이미지 필터를 씌워 사진 꾸미기 앱을 구현 
def show_frame():
    ret, frame = cap.read() 
    frame = cv2.flip(frame, 1) 
        
    if click == 0: 
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)        

    elif click == 1:
        frame = Sketch(frame) 
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)
    elif click == 2:
        frame = Mosaic(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)
    elif click == 3:
        frame = Supreme(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)
    elif click == 4:
        frame = Zombie(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)   
    elif click == 5:
        frame = Corona(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)   
    
    main.after(10, show_frame)
    
show_frame()
main.mainloop()

frame_size =  (640, 480)
